# Building MAML From Scratch

In [ ]:
import numpy as np

## Generate Data Points

Now we define a function called sample_points for generating our input (x,y) pairs. It takes the parameter k as an input which implies number of (x,y) pairs we want to sample.

In [ ]:
def sample_points(k):
    x = np.random.rand(k,50)
    y = np.random.choice([0, 1], size=k, p=[.5, .5]).reshape([-1,1])
    return x,y

The above function returns output as follows,

In [ ]:
x, y = sample_points(10)
print(x[0])
print(y[0])

[0.97701001 0.44817844 0.30013732 0.81729228 0.70880694 0.45539925
 0.12877841 0.61566289 0.41096704 0.67261094 0.91691169 0.15220317
 0.52717107 0.62315087 0.14890789 0.91830606 0.35178133 0.42148066
 0.57827116 0.2260505  0.75068013 0.10678101 0.28770735 0.87530419
 0.41641763 0.56189681 0.67508445 0.87997899 0.13827073 0.62105671
 0.49510489 0.15328504 0.35248309 0.10929488 0.95751146 0.12346789
 0.55446348 0.14252032 0.166051   0.36793247 0.24250467 0.10696335
 0.80372284 0.69391824 0.98584402 0.91588999 0.70855354 0.12828527
 0.23486177 0.15185181]
[1]


## Single Layer Neural Network

For simplicity and better understand, we use a neural network with only single layer for predicting the output. i.e,

a = np.matmul(X, theta)

YHat = sigmoid(a)



__*we use MAML for finding this optimal parameter value theta that is generalizable across tasks. So that
for a new task, we can learn from a few data points in a lesser time by taking very less gradient steps.*__

## MAML

we define a class called MAML where we implement the MAML algorithm. In the \__init__  method we will initialize all the necessary variables. Then we define our sigmoid activation function. Followed by we define our train function.

In [ ]:
class MAML(object):
    def __init__(self):

        #initialize number of tasks i.e number of tasks we need in each batch of tasks
        self.num_tasks = 10

        #number of samples i.e number of shots  -number of data points (k) we need to have in each task
        self.num_samples = 10

        #number of epochs i.e training iterations
        self.epochs = 10000

        #hyperparameter for the inner loop (inner gradient update)
        self.alpha = 0.0001

        #hyperparameter for the outer loop (outer gradient update) i.e meta optimization
        self.beta = 0.0001

        #randomly initialize our model parameter theta
        self.theta = np.random.normal(size=50).reshape(50, 1)

    #define our sigmoid activation function
    def sigmoid(self,a):
        return 1.0 / (1 + np.exp(-a))


    #now let us get to the interesting part i.e training :P
    def train(self):

        #for the number of epochs,
        for e in range(self.epochs):

            self.theta_ = []

            #for task i in batch of tasks
            for i in range(self.num_tasks):

                #sample k data points and prepare our train set
                XTrain, YTrain = sample_points(self.num_samples)

                a = np.matmul(XTrain, self.theta)

                YHat = self.sigmoid(a)

                #since we are performing classification, we use cross entropy loss as our loss function
                loss = ((np.matmul(-YTrain.T, np.log(YHat)) - np.matmul((1 -YTrain.T), np.log(1 - YHat)))/self.num_samples)[0][0]

                #minimize the loss by calculating gradients
                gradient = np.matmul(XTrain.T, (YHat - YTrain)) / self.num_samples

                #update the gradients and find the optimal parameter theta' for each of tasks
                self.theta_.append(self.theta - self.alpha*gradient)


            #initialize meta gradients
            meta_gradient = np.zeros(self.theta.shape)

            for i in range(self.num_tasks):

                #sample k data points and prepare our test set for meta training
                XTest, YTest = sample_points(10)

                #predict the value of y
                a = np.matmul(XTest, self.theta_[i])

                YPred = self.sigmoid(a)

                #compute meta gradients
                meta_gradient += np.matmul(XTest.T, (YPred - YTest)) / self.num_samples


            #update our randomly initialized model parameter theta with the meta gradients
            self.theta = self.theta-self.beta*meta_gradient/self.num_tasks

            if e%1000==0:
                print("Epoch {}: Loss {}\n".format(e,loss))
                print('Updated Model Parameter Theta\n')
                print('Sampling Next Batch of Tasks \n')
                print('---------------------------------\n')

In [ ]:
model = MAML()

In [ ]:
model.train()

Epoch 0: Loss 1.3200553893409737

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 1000: Loss 1.4442779671962247

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 2000: Loss 1.0415903254272676

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 3000: Loss 0.6561256814588772

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 4000: Loss 0.8521443421308212

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 5000: Loss 1.6289202120754773

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 6000: Loss 0.8609538624305001

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 7000: Loss 1.129446096602021

Updated Model Parameter Theta